## Backup your exercises
copy manually your changed exercise files to another directory

## Update your Course Materials!
<code>
cd cbs-python
git checkout -f master
git pull origin master


In case of "merge" problems ...

git fetch --all
git reset --hard origin/master
</code>
<br><br><br>
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Last weeks exercises

###1. Reading CSV files

In [ ]:
csv_file = "../data/genes.csv"

fh = open(csv_file)

fh.readline() # read but ignore the first line

D = {}
for line in fh.readlines():
        fields = line.split('\t')
        if len(fields) != 3: continue
        gene, C1, C2 = fields
        D[gene] = (float(C1), float(C2))

print D



The same code using a dictionary of dictionaries

In [23]:

csv_file = "../data/genes.csv"

fh = open(csv_file)

fh.readline() # read but ignore the first line

D = {}
for line in fh.readlines():
        fields = line.split('\t')
        if len(fields) != 3: continue
        
        gene, C1, C2 = fields
       
        D[gene] = {'C1':float(C1), 'C2':float(C2)}

gene = 'RNF14'
print D[gene]['C1']

fh.close()


53.3653467074


###2. Parsing fasta files

In [24]:

fasta_file = "../data/HIV-1_M-B.fasta"

def parse_fasta(fasta_file):
    entries = []
    entry = {}

    fh = open(fasta_file)
    for line in fh.readlines():
        if line[0] == '>':
            if entry: entries.append(entry)

            fields = line.split('|')
            db, UniqueIdentifier, description = fields
            entry = {}
            entry['DB'] = db[1:]
            entry["AccNum"] = UniqueIdentifier

            d_fields = description.split()

# >db|UniqueIdentifier|EntryName ProteinName OS=OrganismName[ GN=GeneName]
#  PE=ProteinExistence SV=SequenceVersion

#>sp|P03378|ENV_HV1A2 Envelope glycoprotein gp160 OS=Human \
# immunodeficiency virus type 1 group M subtype B (isolate ARV2/SF2) GN=env PE=3 SV=1
            protein_part, organism_part = description.split(' OS=')
            entry["EntryName"] = protein_part.split()[0]
            entry["ProtName"] =  ' '.join(protein_part.split()[1:])
            entry["Organism"] = organism_part.split('PE=')[0].split('GN=')[0].strip()
            entry['Sequence'] = ''

        else:
            entry['Sequence'] = entry['Sequence'] + line.strip()
        

    entries.append(entry)
    return entries

HIV = parse_fasta(fasta_file)
#print 'HIV=', HIV
new_entry =  HIV[0]
print new_entry

{'Sequence': 'MKVKGTRRNYQHLWRWGTLLLGMLMICSATEKLWVTVYYGVPVWKEATTTLFCASDARAYDTEVHNVWATHACVPTDPNPQEVVLGNVTENFNMWKNNMVEQMQEDIISLWDQSLKPCVKLTPLCVTLNCTDLGKATNTNSSNWKEEIKGEIKNCSFNITTSIRDKIQKENALFRNLDVVPIDNASTTTNYTNYRLIHCNRSVITQACPKVSFEPIPIHYCTPAGFAILKCNNKTFNGKGPCTNVSTVQCTHGIRPIVSTQLLLNGSLAEEEVVIRSDNFTNNAKTIIVQLNESVAINCTRPNNNTRKSIYIGPGRAFHTTGRIIGDIRKAHCNISRAQWNNTLEQIVKKLREQFGNNKTIVFNQSSGGDPEIVMHSFNCRGEFFYCNTTQLFNNTWRLNHTEGTKGNDTIILPCRIKQIINMWQEVGKAMYAPPIGGQISCSSNITGLLLTRDGGTNVTNDTEVFRPGGGDMRDNWRSELYKYKVIKIEPLGIAPTKAKRRVVQREKRAVGIVGAMFLGFLGAAGSTMGAVSLTLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARVLAVERYLRDQQLLGIWGCSGKLICTTAVPWNASWSNKSLEDIWDNMTWMQWEREIDNYTNTIYTLLEESQNQQEKNEQELLELDKWASLWNWFSITNWLWYIKIFIMIVGGLVGLRIVFAVLSIVNRVRQGYSPLSFQTRLPVPRGPDRPDGIEEEGGERDRDRSVRLVDGFLALIWEDLRSLCLFSYRRLRDLLLIAARTVEILGHRGWEALKYWWSLLQYWIQELKNSAVSWLNATAIAVTEGTDRVIEVAQRAYRAILHIHRRIRQGLERLLL', 'DB': 'sp', 'AccNum': 'P03378', 'EntryName': 'ENV_HV1A2', 'ProtName': 'Envelope glycoprotein gp160', 'Organism': 'Human immunode

###3. Writing files

In [1]:
def write_codon_table(filename, codon_table):
    fh = open(filename, 'w+')
    for base1 in bases:
        for base2 in bases:
            for base3 in bases:
                codon = base1+base2+base3
                fh.write('%s %s ' % (codon, codon_table[codon]))
            fh.write('\n')
    fh.close()

bases = ['T', 'C', 'A', 'G']
codons = [a+b+c for a in bases for b in bases for c in bases]
amino_acids = 'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'
codon_table = dict(zip(codons, amino_acids))

write_codon_table('mycodon_table.txt', codon_table)

# quickly checking what we have written
print open('mycodon_table.txt').read()

TTT F TTC F TTA L TTG L 
TCT S TCC S TCA S TCG S 
TAT Y TAC Y TAA * TAG * 
TGT C TGC C TGA * TGG W 
CTT L CTC L CTA L CTG L 
CCT P CCC P CCA P CCG P 
CAT H CAC H CAA Q CAG Q 
CGT R CGC R CGA R CGG R 
ATT I ATC I ATA I ATG M 
ACT T ACC T ACA T ACG T 
AAT N AAC N AAA K AAG K 
AGT S AGC S AGA R AGG R 
GTT V GTC V GTA V GTG V 
GCT A GCC A GCA A GCG A 
GAT D GAC D GAA E GAG E 
GGT G GGC G GGA G GGG G 



###4. Sequence sorter

In [ ]:
fh_bacteria = open('../data/eftu_bacteria.faa', 'w')
fh_archaea = open('../data/eftu_archaea.faa', 'w')
fh_eukaryota = open('../data/eftu_eukrayota.faa', 'w')

for line in open('../data/eftu.faa'):
    if line[0] == '>':
        kingdom = line.split()[1]
        if kingdom == 'Bacteria;':
            out_stream = fh_bacteria
        elif kingdom == 'Archaea;':
            out_stream = fh_archaea
        elif kingdom == 'Eukaryota;':
            out_stream = fh_eukaryota

    out_stream.write(line)

###5. DNA translator

In [ ]:
fasta_file = '../data/human_beta_globin.fasta'

def read_fasta(fastafile):
    fh = open(fastafile)
    entries = {}
    for line in fh.readlines():
        if line[0]  == '>':
            seq_name = line[1:].strip()
            entries.setdefault(seq_name,'')
        else:
            entries[seq_name] += line.strip().upper()

    return entries

def translate(sequence, frame=1):
    bases = ['T', 'C', 'A', 'G']
    codons = [a+b+c for a in bases for b in bases for c in bases]
    amino_acids = 'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'
    codon_table = dict(zip(codons, amino_acids))
    aa_seq = []
    for i in range(frame-1, len(sequence), 3):
        codon = sequence[i:i+3]
        if len(codon) !=3: break
        aa_seq.append(codon_table.get(codon, 'X'))
    return ''.join(aa_seq)

def reverse_complement(sequence):
    rseq = sequence[::-1]
    rev_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return ''.join([rev_dict.get(x, 'N') for x in rseq])
                   
            
for name, sequence in read_fasta(fasta_file).items():
    print name
    print sequence
    for i in range(1,4):
        print i, translate(sequence, frame=i)

    r_sequence = reverse_complement(sequence)
    for i in range(1,4):
        print i*-1, translate(r_sequence, frame=i)
    

        


<br><br><br><br><br><br><br><br>

In [26]:
from IPython.core.display import HTML


def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()